# Random Forest using Custom Image

You will be working with the heart disease data to build a random forest model in this segment to predict whether a person is suffering from heart disease or not.
- Heart disease = 0 means that the person does not have heart disease.
- Heart disease = 1 means that the person has heart disease.
- sex = 0 means that the person is female.
- sex = 1 means that the person is male.
- BP: Blood Pressure
- Cholestrol: Cholestrol level

In [1]:
# Import required libraries:
import os
import sys

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import itertools

## Fetch data

In [2]:
# Load the data into dataframe: 
df = pd.read_csv('heart_v2.csv')
df.head()

,age,sex,BP,cholestrol,heart disease
0,70,1,130,322,1
1,67,0,115,564,0
2,57,1,124,261,1
3,64,1,128,263,0
4,74,0,120,269,0


## Prepare data

In [5]:
df.columns[:-1]

Index(['age', 'sex', 'BP', 'cholestrol'], dtype='object')

In [6]:
# Select required columns:
columnReq=[list(df.columns)[-1]]+list(df.columns)[:-1]
columnReq

['heart disease', 'age', 'sex', 'BP', 'cholestrol']

In [7]:
# Update the dataframe:
df=df[columnReq]

In [8]:
# View the updated dataframe:
df.head()

,heart disease,age,sex,BP,cholestrol
0,1,70,1,130,322
1,0,67,0,115,564
2,1,57,1,124,261
3,0,64,1,128,263
4,0,74,0,120,269


In [9]:
# Select all columns except the target column and save it in X:
X=df.iloc[:,1:]
X.head()

,age,sex,BP,cholestrol
0,70,1,130,322
1,67,0,115,564
2,57,1,124,261
3,64,1,128,263
4,74,0,120,269


In [10]:
# Select and save the target variable in y:
y=df.iloc[:,0:1]
y.head()

,heart disease
0,1
1,0
2,1
3,0
4,0


In [12]:
# Import required libraries:
from sklearn.model_selection import train_test_split

In [13]:
# Split the data into train-test split:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=42)
X_train.shape, X_test.shape

((189, 4), (81, 4))

In [14]:
# View the train data:
train=y_train.join(X_train)
train.head()

,heart disease,age,sex,BP,cholestrol
84,0,57,1,110,201
251,0,44,1,130,219
92,1,54,1,124,266
201,1,58,1,125,300
126,1,62,1,120,267


In [15]:
# View test data:
test=y_test.join(X_test)
test.head()

,heart disease,age,sex,BP,cholestrol
30,1,57,1,128,229
116,1,46,1,120,249
79,0,56,1,120,236
127,0,52,0,136,196
196,0,58,0,100,248


In [16]:
# Save train data:
train.to_csv('rftrain.csv',index=False,header=False)

In [17]:
# Save test data: 
test.to_csv('rftest.csv',index=False,header=False)

In [18]:
!/bin/bash ./setup.sh

SageMaker instance route table setup is ok. We are good to go.
SageMaker instance routing for Docker is ok. We are good to go!


In [19]:
import sagemaker
from sagemaker import get_execution_role
from sklearn import preprocessing
from sklearn.metrics import classification_report,confusion_matrix

In [20]:
from sagemaker.sklearn.estimator import SKLearn

In [21]:
sagemaker_session=sagemaker.Session()
role=get_execution_role()

In [22]:
region=sagemaker_session.boto_session.region_name

In [23]:
region

'us-east-1'

In [24]:
train_file="rftrain.csv"
test_file="rftest.csv"
bucket_name="ec2-east-sagemaker-demo"

In [25]:
import re

In [27]:
training_folder=r'BYOA/train'
test_folder=r'BYOA/test'
model_folder=r'BYOA/model/'

In [28]:
training_data_uri=r's3://'+bucket_name+r'/'+training_folder

In [29]:
training_data_uri

's3://ec2-east-sagemaker-demo/BYOA/train'

In [30]:
testing_data_uri=r's3://'+bucket_name+r'/'+test_folder
model_data_uri=r's3://'+bucket_name+r'/'+model_folder

In [32]:
sagemaker_session.upload_data(train_file,
                             bucket=bucket_name,
                             key_prefix=training_folder
                             
                             )

's3://ec2-east-sagemaker-demo/BYOA/train/rftrain.csv'

In [33]:
sagemaker_session.upload_data(test_file,
                             bucket=bucket_name,
                             key_prefix=test_folder
                             
                             )

's3://ec2-east-sagemaker-demo/BYOA/test/rftest.csv'

In [34]:
!pygmentize 'rfcustom.py'

#!/usr/bin/env python
# coding: utf-8

# In[1]:


import argparse
import os
import pandas as pd
import json

from sklearn import ensemble
from sklearn.externals import joblib


# In[ ]:


#Define the Parameter in envirorment variable 


# In[7]:


def parse_args():
    parser = argparse.ArgumentParser()

    # Hyperparameters used for Training
    parser.add_argument('--n_estimators', type=int, default=100)
    parser.add_argument('--max_depth', type=int, default=5)
    
    # path for  Input, output and model directory 
    parser.add_argument('--output-data-dir', type=str, default=os.environ['SM_OUTPUT_DATA_DIR'])
    parser.add_argument('--model-dir', type=str, default=os.environ.get('SM_MODEL_DIR'))
    parser.add_argument('--train', type=str, default=os.environ.get('SM_CHANNEL_TRAINING'))
    parser.add_argument('--test', type=str, default=os.environ.get('SM_CHANNEL_TESTING'))
    
    # Setting variable to host the model
    parser.add_argument('--hosts', type=list, default=json.

In [35]:
instance_type='local'

In [37]:
estimator=SKLearn(entry_point='rfcustom.py',train_instance_type=instance_type,
                 role=role,
                  output_path=model_data_uri,
                  base_job_name='Custom-Random-Forest-v1',
                  hyperparameters={'n_estimators':5,'max_depth':3}
                  
                 )

This is not the latest supported version. If you would like to use version 0.23-1, please add framework_version=0.23-1 to your constructor.


In [38]:
estimator.fit({'training':training_data_uri,'testing':testing_data_uri})

's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


Creating tmp_562d32e_algo-1-f5tiw_1 ... 
Attaching to tmp_562d32e_algo-1-f5tiw_12mdone
algo-1-f5tiw_1  | 2020-08-02 13:20:16,250 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
algo-1-f5tiw_1  | 2020-08-02 13:20:16,253 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-f5tiw_1  | 2020-08-02 13:20:16,265 sagemaker_sklearn_container.training INFO     Invoking user training script.
algo-1-f5tiw_1  | 2020-08-02 13:20:16,404 sagemaker-containers INFO     Module rfcustom does not provide a setup.py. 
algo-1-f5tiw_1  | Generating setup.py
algo-1-f5tiw_1  | 2020-08-02 13:20:16,404 sagemaker-containers INFO     Generating setup.cfg
algo-1-f5tiw_1  | 2020-08-02 13:20:16,404 sagemaker-containers INFO     Generating MANIFEST.in
algo-1-f5tiw_1  | 2020-08-02 13:20:16,404 sagemaker-containers INFO     Installing module with the following command:
algo-1-f5tiw_1  | /miniconda3/bin/python -m pip install . 
algo-1-f5tiw_1  | Processi

algo-1-f5tiw_1  | /miniconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
algo-1-f5tiw_1  |   import imp
algo-1-f5tiw_1  | ####Starting Main Program#########
algo-1-f5tiw_1  | ####ARG######### Namespace(current_host='algo-1-f5tiw', hosts=['algo-1-f5tiw'], max_depth=3, model_dir='/opt/ml/model', n_estimators=5, output_data_dir='/opt/ml/output/data', test='/opt/ml/input/data/testing', train='/opt/ml/input/data/training')
algo-1-f5tiw_1  | #######Successfully Loaded Train
algo-1-f5tiw_1  | ####### Test Successfully Loaded
algo-1-f5tiw_1  | ####### Model Started Training
algo-1-f5tiw_1  | Training Accuracy: 0.730
algo-1-f5tiw_1  | Testing Accuracy: 0.630
algo-1-f5tiw_1  | 2020-08-02 13:20:18,562 sagemaker-containers INFO     Reporting training SUCCESS
tmp_562d32e_algo-1-f5tiw_1 exited with code 0
Aborting on container

In [40]:
predictor=estimator.deploy(instance_type=instance_type,initial_instance_count=1)

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


Attaching to tmppo8jm0h3_algo-1-sjopc_1
algo-1-sjopc_1  | Processing /opt/ml/code
algo-1-sjopc_1  | Building wheels for collected packages: rfcustom
algo-1-sjopc_1  |   Building wheel for rfcustom (setup.py) ... done
algo-1-sjopc_1  |   Created wheel for rfcustom: filename=rfcustom-1.0.0-py2.py3-none-any.whl size=6401 sha256=fdf0eae7a7ece2830cde105ca0a16d1fbb80e043433e49dd2000412944f1b332
algo-1-sjopc_1  |   Stored in directory: /tmp/pip-ephem-wheel-cache-_ooei1xf/wheels/35/24/16/37574d11bf9bde50616c67372a334f94fa8356bc7164af8ca3
algo-1-sjopc_1  | Successfully built rfcustom
algo-1-sjopc_1  | Installing collected packages: rfcustom
algo-1-sjopc_1  | Successfully installed rfcustom-1.0.0
algo-1-sjopc_1  | /miniconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
algo-1-sjopc_1  |   import imp
algo-1-sjopc_1  | [2020-

In [41]:
dfinf=pd.read_csv(test_file,names=columnReq)

In [42]:
dfinf.head()

,heart disease,age,sex,BP,cholestrol
0,1,57,1,128,229
1,1,46,1,120,249
2,0,56,1,120,236
3,0,52,0,136,196
4,0,58,0,100,248


In [43]:
xtest=dfinf.iloc[:,1:]

In [44]:
xtest.head()

,age,sex,BP,cholestrol
0,57,1,128,229
1,46,1,120,249
2,56,1,120,236
3,52,0,136,196
4,58,0,100,248


In [45]:
pred=predictor.predict(xtest)

algo-1-sjopc_1  | 2020-08-02 13:29:53,998 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
algo-1-sjopc_1  | /miniconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
algo-1-sjopc_1  |   import imp
algo-1-sjopc_1  | ###### Uploaded model########
algo-1-sjopc_1  | 172.18.0.1 - - [02/Aug/2020:13:29:54 +0000] "POST /invocations HTTP/1.1" 200 776 "-" "-"


In [46]:
pred

array([1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0])

In [48]:
ytest=dfinf.iloc[:,0:1]

In [50]:
ytest=ytest.values

In [51]:
ytest

array([[1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0]])

In [52]:
confusion_matrix(ytest,pred)

array([[37, 12],
       [18, 14]])

In [53]:
classification_report(ytest,pred)

'              precision    recall  f1-score   support\n\n           0       0.67      0.76      0.71        49\n           1       0.54      0.44      0.48        32\n\n    accuracy                           0.63        81\n   macro avg       0.61      0.60      0.60        81\nweighted avg       0.62      0.63      0.62        81\n'

In [ ]:
!/bin/bash ./setup.sh

In [ ]:
# Import required libraries:
import sagemaker
from sagemaker import get_execution_role
from sklearn import preprocessing
from sklearn.metrics import classification_report, confusion_matrix

# SageMaker SKLearn Estimator
from sagemaker.sklearn.estimator import SKLearn

In [ ]:
# Create SageMaker session:
sagemaker_session = sagemaker.Session()

role = get_execution_role()
region = sagemaker_session.boto_session.region_name

In [ ]:
# Specify your bucket name:
train_file='rftrain.csv'
test_file='rftest.csv'
bucket_name = 'ec2-east-sagemaker-demo'

# Specify file names:
training_folder = r'BYOA/train'
test_folder = r'BYOA/test'
model_folder = r'BYOA/model/'

# Save files to S3
training_data_uri = r's3://' + bucket_name + r'/' + training_folder
testing_data_uri = r's3://' + bucket_name + r'/' + test_folder
model_data_uri = r's3://' + bucket_name + r'/' + model_folder

In [ ]:
# Upload train data:
sagemaker_session.upload_data(train_file,
                              bucket=bucket_name, 
                              key_prefix=training_folder)

In [ ]:
# Upload test data:
sagemaker_session.upload_data(test_file, 
                              bucket=bucket_name, 
                              key_prefix=test_folder)

In [ ]:
!pygmentize 'rfcustom.py'

In [ ]:
# set instance_type='ml.m5.xlarge'
instance_type='local'

In [ ]:
# Build instance:
estimator = SKLearn(entry_point='rfcustom.py',
                    train_instance_type= instance_type,                     
                    role=role, 
                    output_path=model_data_uri,
                    base_job_name='Custom Random-forest',
                    hyperparameters={'n_estimators': 5,'max_depth':3})

## Train the model

In [ ]:
# Train the model:
estimator.fit({'training':training_data_uri,'testing':testing_data_uri})

## Deploy the Model

In [ ]:
# Deploy the model:
predictor = estimator.deploy(initial_instance_count=1, 
                           instance_type=instance_type)

In [ ]:
# Read test file:
dftst = pd.read_csv(test_file,names=columnReq)
dftst.head()

In [ ]:
# View test data for X:
xtest=dftst.iloc[:,1:]
xtest.head()

In [ ]:
# View y:
ytest=dftst.iloc[:,0:1]

In [ ]:
# Import required libraries:
from sklearn import preprocessing
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# Predict:
k=predictor.predict(xtest)

In [ ]:
# Print Confusion Matrix:
confusion_matrix(ytest,k)

In [ ]:
# Print classification report:
classification_report(ytest,k)